# Deep learning A-Z : Building a Boltzmann Machine

<p align="justify">
This notebook is my response to the fifth homework of the course called *Deep Learning A-Z™: Hands-On Artificial Neural Networks* accessible here : https://www.udemy.com/deeplearning/
</p>
<p align="justify">
In this notebook, we are going to build two recommended systems which will tell for each user if this user liked or no a movie. The first recommended system will have a binary output to tell if the user liked or not and the second one will ouput a rating from 1 to five. We are going to build this recommended system using Boltzmann machines built with pytorch.
</p>

### Imports

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### 1. Data preprocessing

In [3]:
base_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('__file__')))), 
                          'ressources/Boltzmann_Machines/')

movies = pd.read_csv(base_path + 'ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv(base_path + 'ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv(base_path + 'ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

movies.head()


,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


For the ratings dataset, the first col is the user id, the second is the genre and the third one is the grade. Fourth one is the zip code.

In [4]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


For the ratings dataset, the first col is the user id, the second is the movie that this user rated and the third one is the grade. Fourth one is just a timestamp.

In [5]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# cols are the same as for ratings dataset
training_set = pd.read_csv(base_path + 'ml-100k/u1.base', delimiter='\t')
test_set = pd.read_csv(base_path + 'ml-100k/u1.test', delimiter='\t')
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [7]:
# convert data to array
training_set = np.array(training_set, dtype='int')
test_set = np.array(test_set, dtype='int')
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [8]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [9]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

Tensors are array with only one data type. So Torch tensors are just multi-dimentionnal arrays exactly as tensorflow tensors.

In [10]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

Now, our data are tensors :

In [11]:
training_set

tensor([[0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

### 2. Building the Boltzmann Machine for binary outputs

First, as we want to predict binary output, we need to change our input ratings into a binary rating. Indeed, predicted binary rating will be calculated from input rating so it must be binary. Moreover, as 0 currently means that a user doesn't rated a movie, we need to change this value (to -1) because for binary output we need the 0 value.

In [ ]:
training_set[training_set == 0] = -1 # replace 0 by -1 for non rated movies
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

Now, let's create the model !

### 3. Building the Boltzmann Machine for ratings outputs